In [1]:
import lzma
import pickle

import dill
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
from jitr.optical_potentials import chuq

In [3]:
import rxmc

Using database version X4-2024-12-31 located in: /mnt/home/beyerkyl/x4db/unpack_exfor-2024/X4-2024-12-31


## read in CHUQ parameter samples

In [4]:
neutron = (1, 0)
proton = (1, 1)

In [5]:
chuq_params = chuq.get_samples_federal()

In [6]:
nparams = len(chuq_params[0])

## read in observations and models

In [7]:
from pathlib import Path

evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")

output_dir = Path("./chuq_propagation/")
output_dir.mkdir(parents=True, exist_ok=True)

In [8]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

In [9]:
with open(model_dir / "chuq_nn_model.pkl", "rb") as f:
    nn_model = dill.load(f)

In [10]:
with open(model_dir / "chuq_pp_model.pkl", "rb") as f:
    pp_model = dill.load(f)

In [11]:
def propagate(samples):
    yth_nn = []
    for entry_id, obs in tqdm(nn_elastic_obs.items()):
        for x in obs:
            yth_nn.append(
                np.array(
                    [
                        nn_model.visualizable_model_prediction(x, *p)
                        for p in samples
                    ]
                )
            )
    yth_pp = []
    for entry_id, obs in tqdm(pp_elastic_obs.items()):
        for x in obs:
            yth_pp.append(
                np.array(
                    [
                        pp_model.visualizable_model_prediction(x, *p)
                        for p in samples
                    ]
                )
            )
    return yth_nn, yth_pp

## propagate CHUQ into constraint observables

In [12]:
yth_nn, yth_pp = propagate(chuq_params)

100%|██████████| 55/55 [01:09<00:00,  1.27s/it]


## write posterior predictive distributions to disk

In [13]:
with open(output_dir / "nn_chuq_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [14]:
with open(output_dir / "pp_chuq_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)

## Propagate posteriors from our calibration

In [18]:
results_1 = np.load("../calibration/chuq_cal_prod_1/result.npz")
results_01 = np.load("../calibration/chuq_cal_prod_01/result.npz")
results_001 = np.load("../calibration/chuq_cal_prod_001/result.npz")

n_chains, n_steps, n_params = results_1["model_chains"].shape
(n_chains, n_steps, n_params)

(64, 100000, 22)

In [19]:
def draw_samples(x, low_cut=None, high_cut=None, n_samples=None):
    chains, steps, n_params = x.shape
    if low_cut is None:
        low_cut = 0
    if high_cut is None:
        high_cut = steps - 1
    samples = x[:, low_cut:high_cut, ...].reshape(
        (chains * (high_cut - low_cut), n_params)
    )
    if n_samples is None:
        return samples
    else:
        return samples[
            np.random.choice(np.arange(chains * (high_cut - low_cut)), n_samples), ...
        ]

In [20]:
posterior_1 = draw_samples(results_1["model_chains"], low_cut=50000, n_samples=1000)
posterior_01 = draw_samples(results_1["model_chains"], low_cut=50000, n_samples=1000)
posterior_001 = draw_samples(results_1["model_chains"], low_cut=50000, n_samples=1000)

In [21]:
yth_nn, yth_pp = propagate(posterior_1)
with open(output_dir / "nn_chuq_cal_1.pkl", "wb") as f:
    pickle.dump(yth_nn, f)
with open(output_dir / "pp_chuq_cal_1.pkl", "wb") as f:
    pickle.dump(yth_pp, f)

100%|██████████| 55/55 [05:30<00:00,  6.01s/it]


In [22]:
yth_nn, yth_pp = propagate(posterior_01)
with open(output_dir / "nn_chuq_cal_01.pkl", "wb") as f:
    pickle.dump(yth_nn, f)
with open(output_dir / "pp_chuq_cal_01.pkl", "wb") as f:
    pickle.dump(yth_pp, f)

100%|██████████| 55/55 [05:29<00:00,  6.00s/it]


In [23]:
yth_nn, yth_pp = propagate(posterior_001)
with open(output_dir / "nn_chuq_cal_001.pkl", "wb") as f:
    pickle.dump(yth_nn, f)
with open(output_dir / "pp_chuq_cal_001.pkl", "wb") as f:
    pickle.dump(yth_pp, f)

100%|██████████| 55/55 [05:30<00:00,  6.01s/it]
